In [1]:
import numpy as np
import pandas as pd
from subprocess import Popen, PIPE, DEVNULL
import glob

In [2]:
# constants
num_chrom = 9
max_chrom_len = 10000000
num_rows = np.logspace(1, 7, base=10).astype(int)

In [3]:
def random_sort(i, seed, size, group):
    p1 = Popen([
        "gia", "random", f"-n{i}", f"-l{size}", f"-m{max_chrom_len}", f"-c{num_chrom}", f"-s{seed}"
    ], stdout=PIPE)
    p2 = Popen([
        "gia", "sort", f"-odata/random_{i}_{group}.bed"
    ], stdin=p1.stdout)
    p2.communicate()

def get_mem(args: list):
    p = Popen(
        ["gtime", "-f'%M'"] + args,
        stdout=DEVNULL, stderr=PIPE
    )
    stdout, stderr = p.communicate()
    return stderr.decode('utf-8').strip().replace("'", "") 

def gia_intersect(i):
    args = [
        "gia", "intersect", 
        "-a", f"data/random_{i}_a.bed", 
        "-b", f"data/random_{i}_b.bed", 
    ]
    return get_mem(args)

def gia_intersect_stream(i):
    args = [
        "gia", "intersect", 
        "-a", f"data/random_{i}_a.bed", 
        "-b", f"data/random_{i}_b.bed", 
        "-S",
    ]
    return get_mem(args)

def bedtools_intersect(i):
    args = [
        "bedtools", "intersect",
        "-a", f"data/random_{i}_a.bed", 
        "-b", f"data/random_{i}_b.bed",
    ]
    return get_mem(args)

def bedtools_intersect_stream(i):
    args = [
        "bedtools", "intersect",
        "-a", f"data/random_{i}_a.bed", 
        "-b", f"data/random_{i}_b.bed",
        "-sorted",
    ]
    return get_mem(args)

def bedops_intersect(i):
    args = [
        "bedops", "-i",
        f"data/random_{i}_a.bed", 
        f"data/random_{i}_b.bed",
    ]
    return get_mem(args)

In [4]:
for i in num_rows:
    random_sort(i, 0, 150, "a")
    random_sort(i, 1, 300, "b")

In [ ]:
frame = []

for i in num_rows:
    frame.append({
        "tool": "gia",
        "method": "inplace",
        "num_rows": i,
        "mem": gia_intersect(i)
    })
    frame.append({
        "tool": "gia",
        "method": "stream",
        "num_rows": i,
        "mem": gia_intersect_stream(i)
    })
    frame.append({
        "tool": "bedtools",
        "method": "inplace",
        "num_rows": i,
        "mem": bedtools_intersect(i)
    })
    frame.append({
        "tool": "bedtools",
        "method": "stream",
        "num_rows": i,
        "mem": bedtools_intersect_stream(i)
    })
    frame.append({
        "tool": "bedops",
        "method": "stream",
        "num_rows": i,
        "mem": bedops_intersect(i)
    })

frame = pd.DataFrame(frame)
frame["mem"] = frame["mem"].astype(int)
frame

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
frame["named"] = frame.tool + "_" + frame.method

In [ ]:
plt.figure(figsize=(8, 4), dpi=300)
sns.lineplot(
    data=frame,
    x="num_rows",
    y="mem",
    hue="tool",
    style="method",
    palette="Set1",
    hue_order=["gia", "bedops", "bedtools"],
    style_order=["inplace", "stream"]
)
plt.yscale("log")
plt.xscale("log")
plt.ylabel("Peak Memory Usage (kB)")
plt.xlabel("Number of BED Intervals")
plt.title("Memory Usage : Intersection")
plt.tight_layout()
plt.savefig("figures/intersection_memory.svg")
plt.savefig("figures/intersection_memory.png")
plt.show()

In [ ]:
frame[frame.method == "stream"].groupby("tool").apply(lambda x: x.mem.mean())

In [ ]:
def classify_memory_usage(x):
    if "bedops" in x:
        return "stream"
    if "gia" in x:
        if "-S" in x:
            return "stream"
        else:
            return "inplace"
    if "bedtools" in x:
        if "-sorted" in x:
            return "stream"
        else:
            return "inplace"

timing_frame = []
for fn in glob.glob("results/intersect_range_*.csv"):
    num_iv = int(fn.split("_")[-1].split(".")[0])
    try:
        subframe = pd.read_csv(fn)
        subframe["num_rows"] = num_iv
        timing_frame.append(subframe)
    except:
        print(fn)
    
timing_frame = pd.concat(timing_frame)
timing_frame["tool"] = timing_frame.command.apply(lambda x: x.split(" ")[0])
timing_frame["method"] = timing_frame.command.apply(lambda x: classify_memory_usage(x))
timing_frame["named"] = frame.tool + "_" + frame.method
timing_frame

In [ ]:
plt.figure(figsize=(8,4), dpi=300)
sns.lineplot(
    data=timing_frame,
    x="num_rows",
    y="mean",
    hue="tool",
    style="method",
    palette="Set1",
    hue_order=["gia", "bedops", "bedtools"], 
    style_order=["inplace", "stream"]

)
plt.yscale("log")
plt.xscale("log")
plt.ylabel("Mean Elapsed Time (s)")
plt.xlabel("Number of BED Intervals")
plt.title("Time Elapsed: Intersection")
plt.tight_layout()
plt.savefig("figures/intersection_compute.svg")
plt.savefig("figures/intersection_compute.png")
plt.show()